### Проверка API, тестовые запросы

In [24]:
import requests
import json
import os

file_path = os.path.join('..', '.gigachat', 'gigachat.json')

with open(file_path, 'r') as file:
    config = json.load(file)
    auth_key = config.get('authorization_key')

url = 'https://ngw.devices.sberbank.ru:9443/api/v2/oauth'

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'application/json',
    'RqUID': '0511d05f-eefb-4900-b0ca-047632578c01',
    'Authorization': f'Basic {auth_key}'
}

data = {
    'scope': 'GIGACHAT_API_PERS'
}

response = requests.request("POST", url, headers=headers, data=data, verify=False).json()
access_token = response.get('access_token')

In [3]:
import requests
import json

def chat_request(access_token, data = {}, url = "https://gigachat.devices.sberbank.ru/api/v1/models"):  
  headers = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {access_token}'
  }

  return requests.request("GET", url, headers=headers, data=data, verify=False).json()

chat_request(access_token)

/home/denstrygin/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'object': 'list',
 'data': [{'id': 'GigaChat',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-2',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-2-Max',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-2-Max-preview',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-2-Pro',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-2-Pro-preview',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-2-preview',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-Max',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-Max-preview',
   'object': 'model',
   'owned_by': 'salutedevices',
   'type': 'chat'},
  {'id': 'GigaChat-Plus',
   'object': 'model',
  

In [6]:
import requests
import json
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def gigachat_request(access_token, message, model_name="GigaChat-2", max_tokens=512, repetition_penalty=1.0, stream=False, url="https://gigachat.devices.sberbank.ru/api/v1/chat/completions"):
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    data = {
        "model": model_name,
        "messages": [
            {
                "role": "user",
                "content": f"Answer on english language. {message}"
            }
        ],
        "n": 1,
        "stream": stream,
        "max_tokens": max_tokens,
        "repetition_penalty": repetition_penalty,
        "update_interval": 0
    }

    response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка: {response.status_code}")
        print(response.text)
        return None

print("Пример генерации квеста")
print(gigachat_request(access_token, message = "Generate a quest for a text-based video game. The quest should be engaging and include a compelling backstory, clear objectives, interesting characters, and meaningful challenges. It should also provide opportunities for player choice and multiple outcomes."))
print("Пример генерации локации")
print(gigachat_request(access_token, message = "Generate a detailed description of a game location. The location should be immersive, with a unique atmosphere, and include environmental details, potential points of interest, and hints of lore. The description should make the player feel like they are exploring a mysterious and engaging world."))
print("Пример генерации диалога")
print(gigachat_request(access_token, message = "In a smoky tavern, the player approaches a mysterious hooded figure known as \"The Whisperer,\" a former assassin with a haunted past."))

Пример генерации квеста
{'choices': [{'message': {'content': '# The Frozen Dream: A Quest for the Aurora’s Tear\n\n## **Backstory**\n\nIn the ancient realm of Néos Kryos, a timeless battle rages between the forces of light and darkness. Long ago, the Aurora, a goddess of hope and beauty, created a magical artifact known as the Aurora’s Tear. This glowing crystal, capable of purifying darkness and dispelling despair, was hidden deep within the mountains of the Northern Reaches.\n\nBut centuries ago, the crystal was stolen by the shadowy overlord, Valgor the Betrayer, who sought to use its power to dominate the world. Now, the people of Néos Kryos live in fear, as Valgor’s reign of darkness grows stronger with each passing day.\n\n## **Objective**\n\nYou are an adventurer known only as the Chosen One. You have been called to Néos Kryos by the spirit of the Aurora herself. Your mission is to recover the Aurora’s Tear and restore balance to the realm.\n\n## **Characters**\n\n1. **The Chose

### Работа с данными

In [ ]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

def load_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def process_text(text):
    doc = nlp(text)
    return doc

def extract_dialogues(text):
    """
    Извлекает диалоги из текста, группируя реплики в рамках одного диалога.
    Диалог считается завершенным, если между репликами появляется абзац без реплик.
    """
    dialog_pattern = r"([\"'“”‘’])\s*(?!\w+'?\w*\1)(.*?)\1"
    paragraphs = text.split("\n")
    
    dialogue_blocks = []
    current_dialog = []

    for paragraph in paragraphs:
        paragraph = paragraph.strip()
        if not paragraph:
            # Завершаем текущий диалог, если он не пустой
            if current_dialog:
                block = " ".join(current_dialog).strip()
                if block:
                    dialogue_blocks.append(block)
                current_dialog = []
            continue
        
        # Находим все реплики в абзаце
        matches = re.findall(dialog_pattern, paragraph)
        
        # Фильтруем только настоящие реплики
        for match in matches:
            content = match[1].strip()
            if len(content.split()) > 2:
                current_dialog.append(content)
    
    # Добавляем последний блок, если он не пустой
    if current_dialog:
        block = " ".join(current_dialog).strip()
        if block:
            dialogue_blocks.append(block)

    # Возвращаем только непустые блоки
    return [d for d in dialogue_blocks if d.strip()]

def extract_locations(text):
    """
    Извлекает описания локаций из текста, ищет ключевые слова и контексты, связанные с локациями.
    """
    doc = nlp(text)
    
    # Ключевые слова для поиска локаций
    location_keywords = ["hill", "forest", "river", "mountain", "town", "village", "castle", "cave"]
    
    location_descriptions = []
    
    for sent in doc.sents:
        if any(keyword in sent.text.lower() for keyword in location_keywords):
            location_descriptions.append(sent.text.strip())
    
    return location_descriptions

def extract_quests(text):
    """
    Извлекает квесты из текста, основываясь на ключевых словах, таких как "find", "rescue", "search".
    """
    quest_keywords = ["find", "go", "rescue", "search", "quest", "help", "bring", "retrieve", "save"]
    doc = nlp(text)
    
    quest_sentences = []
    for sent in doc.sents:
        if any(keyword in sent.text.lower() for keyword in quest_keywords):
            quest_sentences.append(sent.text)
    return quest_sentences

In [8]:
import os

books_dir = os.path.join('datasets', 'books')
parsed_books_dir = os.path.join('datasets', 'parsed_books')

os.makedirs(parsed_books_dir, exist_ok=True)

def save_list_to_file(data_list, book_dir, filename):
    os.makedirs(book_dir, exist_ok=True)
    with open(os.path.join(book_dir, filename), 'w', encoding='utf-8') as f:
        for item in data_list:
            f.write(item.strip() + '\n\n')

for filename in os.listdir(books_dir):
    if filename.endswith('.txt'):
        book_name = os.path.splitext(filename)[0]
        file_path = os.path.join(books_dir, filename)
        
        text = load_text_from_file(file_path)

        dialogues = extract_dialogues(text)
        locations = extract_locations(text)
        quests = extract_quests(text)

        book_output_dir = os.path.join(parsed_books_dir, book_name)

        save_list_to_file(dialogues, book_output_dir, 'dialogues.txt')
        save_list_to_file(locations, book_output_dir, 'locations.txt')
        save_list_to_file(quests, book_output_dir, 'quests.txt')

### RAG
1. Создание индексов
2. Создание ретривера и тестирование ответов

In [1]:
import os
import faiss
import json
from sentence_transformers import SentenceTransformer

PARSED_BOOKS_DIR = os.path.join("datasets", "parsed_books")
INDEX_DIR = os.path.join("datasets", "indexes")

os.makedirs(INDEX_DIR, exist_ok=True)

embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

texts = []
metadata = []

for book_name in os.listdir(PARSED_BOOKS_DIR):
    book_dir = os.path.join(PARSED_BOOKS_DIR, book_name)
    if os.path.isdir(book_dir):
        for filename in os.listdir(book_dir):
            filepath = os.path.join(book_dir, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                data = file.read().split('\n\n')
                for passage in data:
                    passage = passage.strip()
                    if passage:
                        texts.append(passage)
                        metadata.append({
                            "book": book_name,
                            "file": filename,
                            "text": passage
                        })

print("Создание эмбеддингов")
embeddings = embedder.encode(texts, show_progress_bar=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

faiss.write_index(index, os.path.join(INDEX_DIR, "faiss_index"))
with open(os.path.join(INDEX_DIR, "metadata.json"), 'w', encoding='utf-8') as meta_file:
    json.dump(metadata, meta_file)

[2025-05-13 11:41:07,383] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/denstrygin/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/denstrygin/miniconda3/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12.6/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/denstrygin/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.6/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/denstrygin/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.6/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/denstrygin/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.6/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/denstrygin/miniconda3/compiler_compat/ld: /usr/local/cuda-12.6/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/denstrygin/miniconda3/compiler_

Создание эмбеддингов


Batches:   0%|          | 0/286 [00:00<?, ?it/s]

In [ ]:
import os
import faiss
import json
import requests
import urllib3
from sentence_transformers import SentenceTransformer
from langchain.retrievers import MultiQueryRetriever
from langchain_core.retrievers import BaseRetriever
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from pydantic import PrivateAttr
import numpy as np

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

INDEX_DIR = os.path.join("datasets", "indexes")
NUM_RESULTS = 5
ACCESS_TOKEN = access_token
GIGACHAT_MODEL_NAME = "GigaChat-2"

print("Загрузка индексов и метаданных")
index = faiss.read_index(f"{INDEX_DIR}/faiss_index")
with open(f"{INDEX_DIR}/metadata.json", 'r', encoding='utf-8') as meta_file:
    metadata = json.load(meta_file)

# Используем Sentence Transformers для создания эмбеддингов
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def gigachat_request(access_token, message, model_name="GigaChat-2", max_tokens=512, repetition_penalty=1.0, stream=False, url="https://gigachat.devices.sberbank.ru/api/v1/chat/completions"):
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    data = {
        "model": model_name,
        "messages": [
            {
                "role": "user",
                "content": f"Answer on english language. {message}"
            }
        ],
        "n": 1,
        "stream": stream,
        "max_tokens": max_tokens,
        "repetition_penalty": repetition_penalty,
        "update_interval": 0
    }

    response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"].strip()
    else:
        print(f"Ошибка: {response.status_code}")
        print(response.text)
        return None

prompt_template = PromptTemplate.from_template(
    "Generate 5 different ways to ask the following question in a more detailed and creative way: {question}"
)
llm_runnable = RunnableLambda(lambda prompt: gigachat_request(ACCESS_TOKEN, prompt))
llm_chain = prompt_template | llm_runnable | StrOutputParser()

class FaissRetriever(BaseRetriever):
    _index = PrivateAttr()
    _metadata = PrivateAttr()
    _embedder = PrivateAttr()
    _num_results = PrivateAttr()

    def __init__(self, index, metadata, embedder, num_results=5):
        super().__init__()
        self._index = index
        self._metadata = metadata
        self._embedder = embedder
        self._num_results = num_results

    def _get_relevant_documents(self, query):
        # Генерируем эмбеддинг для запроса
        query_embedding = self._embedder.encode([query]).astype(np.float32)
        # Ищем ближайшие документы
        _, indices = self._index.search(query_embedding, self._num_results)
        # Формируем список документов
        documents = [Document(page_content=self._metadata[i]["text"], metadata=self._metadata[i]) for i in indices[0]]
        return documents

faiss_retriever = FaissRetriever(index=index, metadata=metadata, embedder=embedder, num_results=NUM_RESULTS)

multi_query_retriever = MultiQueryRetriever(
    retriever=faiss_retriever,
    llm_chain=llm_chain,
    num_queries=5
)

def retrieve(query, method="hyde"):
    """
    Функция поиска фрагментов в индексе.
    """
    if method == "hyde":
        hyde_prompt = f"Imagine a detailed, hypothetical answer to the following question: {query}"
        hyde_query = gigachat_request(ACCESS_TOKEN, hyde_prompt)
        print("\n[HyDE Prompt]:", hyde_prompt)
        print("\n[HyDE Generation]:", hyde_query)
        query_embedding = embedder.encode([hyde_query]).astype(np.float32)
        
    elif method == "multi_query":
        multi_queries = llm_chain.invoke({"question": query}).split("\n")
        print("\n[MultiQuery Generation]:", multi_queries)
        query_embeddings = embedder.encode(multi_queries).astype(np.float32)
        query_embedding = query_embeddings.mean(axis=0).reshape(1, -1)

    else:
        raise ValueError(f"Unknown method: {method}")

    # Ищем ближайшие фрагменты
    _, indices = index.search(query_embedding, NUM_RESULTS)
    results = [metadata[idx] for idx in indices[0]]
    passages = [metadata[idx]["text"] for idx in indices[0]]

    return passages, results

# Примеры использования
queries = [
    "Generate a location in the style of a futuristic war with aliens.",
    "Create a dialogue in the style of medieval conversations.",
    "Generate a quest in the style of a fantasy adventure."
]

for method in ["hyde", "multi_query"]:
    print(f"\n=== {method.upper()} ===")
    for query in queries:
        print(f"\n[QUERY]: {query}")
        passages, meta = retrieve(query, method=method)
        for passage in passages:
            print("-", passage)


Загрузка индексов и метаданных


No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.



=== HYDE ===

[QUERY]: Generate a location in the style of a futuristic war with aliens.

[HyDE Prompt]: Imagine a detailed, hypothetical answer to the following question: Generate a location in the style of a futuristic war with aliens.

[HyDE Generation]: ### Hypothetical Location: The Battle of Novaya Zemlya

**Location Description:**

**Name:** Novaya Zemlya
**Region:** Arctic Circle, Siberia
**Environment:** A desolate, icy archipelago, where winds howl relentlessly and snow drifts across frozen plains. The landscape is rugged, with jagged mountains, glaciers, and vast stretches of ice-covered tundra.

**Historical Context:**

Novaya Zemlya has long been a strategic location for Russia, due to its strategic position near the North Pole. During the Cold War, it was a testing ground for nuclear weapons and a place where the Soviet Union conducted numerous top-secret experiments. After the fall of the Soviet Union, the area was largely abandoned, leaving it in a state of neglect and